# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [33]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys1 import geoapify_key

In [34]:
#constant
BASE_URL = "https://api.geoapify.com/v2/places"
HTTP_STATUS_OK = 200
HTTP_STATUS_NOT_FOUND = 404

In [35]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,stanley,54.8680,-1.6985,11.66,83,75,10.80,GB,1707073227
1,1,chonchi,-42.6128,-73.8092,24.82,30,3,0.97,CL,1707072982
2,2,togur,58.3631,82.8265,-4.36,96,100,4.47,RU,1707073490
3,3,palikir - national government center,6.9248,158.1611,26.41,83,100,4.63,FM,1707073312
4,4,port-aux-francais,-49.3500,70.2167,5.05,72,7,10.06,TF,1707072968


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [36]:
%%capture --no-display
#tiles = "EsriImagery"

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = True,
    frame_width = 800,
    frame_height = 500,
    size= 'Humidity',
    c= 'Humidity',
    cmap= 'rainbow',
    alpha= 0.6
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [37]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] < 10) &
    (city_data_df['Max Temp'] >= 21) & (city_data_df['Max Temp'] < 28)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,chonchi,-42.6128,-73.8092,24.82,30,3,0.97,CL,1707072982
5,5,sur,22.5667,59.5289,21.43,62,4,2.51,OM,1707073367
46,46,saint-pierre,-21.3393,55.4781,22.82,73,0,2.57,RE,1707073494
49,49,tocopilla,-22.0920,-70.1979,22.33,77,6,2.33,CL,1707073313
58,58,hermanus,-34.4187,19.2345,21.00,88,4,1.03,ZA,1707072975
178,178,ratnagiri,16.9833,73.3000,24.43,67,0,1.96,IN,1707073514
192,192,nova sintra,14.8667,-24.7167,21.52,69,0,3.85,CV,1707072979
224,224,salalah,17.0151,54.0924,23.72,43,0,2.13,OM,1707073521
244,244,cabo san lucas,22.8909,-109.9124,24.49,56,0,2.24,MX,1707073053
259,259,antofagasta,-23.6500,-70.4000,27.60,66,7,2.93,CL,1707073021


### Step 3: Create a new DataFrame called `hotel_df`.

In [38]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,chonchi,-42.6128,-73.8092,24.82,30,3,0.97,CL,1707072982,
5,5,sur,22.5667,59.5289,21.43,62,4,2.51,OM,1707073367,
46,46,saint-pierre,-21.3393,55.4781,22.82,73,0,2.57,RE,1707073494,
49,49,tocopilla,-22.0920,-70.1979,22.33,77,6,2.33,CL,1707073313,
58,58,hermanus,-34.4187,19.2345,21.00,88,4,1.03,ZA,1707072975,
178,178,ratnagiri,16.9833,73.3000,24.43,67,0,1.96,IN,1707073514,
192,192,nova sintra,14.8667,-24.7167,21.52,69,0,3.85,CV,1707072979,
224,224,salalah,17.0151,54.0924,23.72,43,0,2.13,OM,1707073521,
244,244,cabo san lucas,22.8909,-109.9124,24.49,56,0,2.24,MX,1707073053,
259,259,antofagasta,-23.6500,-70.4000,27.60,66,7,2.93,CL,1707073021,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories": categories,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Make and API request using the params dictionaty
    response = requests.get(BASE_URL, params=params)

    if response.status_code == HTTP_STATUS_OK:
        # Convert the API response JSON format string to a python dictionary
        name_address = response.json()
        
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
            print(response.status_code)
            
        # Log the search results
        print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

    elif response.status_code == HTTP_STATUS_NOT_FOUND:
        # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    else:
        raise ValueError(f"Invalid Status Code {response.status_code} for {response.url}") 


# Display sample data
hotel_df

Starting hotel search


chonchi - nearest hotel: Hotel Huildín
sur - nearest hotel: Sur Hotel
saint-pierre - nearest hotel: Tropic Hotel
tocopilla - nearest hotel: hotel Isidora
hermanus - nearest hotel: Aloe guest house
ratnagiri - nearest hotel: Vihar
nova sintra - nearest hotel: Residência Ka Dencho
salalah - nearest hotel: Muscat International Hotel
cabo san lucas - nearest hotel: Comfort Rooms
antofagasta - nearest hotel: Hotel Colón
sompeta - nearest hotel: Haritha Beach Resort
al ghayzah - nearest hotel: فندق تاج العرب
200
howard springs - nearest hotel: No hotel found
200
kununurra - nearest hotel: No hotel found
200
birao - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,chonchi,-42.6128,-73.8092,24.82,30,3,0.97,CL,1707072982,Hotel Huildín
5,5,sur,22.5667,59.5289,21.43,62,4,2.51,OM,1707073367,Sur Hotel
46,46,saint-pierre,-21.3393,55.4781,22.82,73,0,2.57,RE,1707073494,Tropic Hotel
49,49,tocopilla,-22.0920,-70.1979,22.33,77,6,2.33,CL,1707073313,hotel Isidora
58,58,hermanus,-34.4187,19.2345,21.00,88,4,1.03,ZA,1707072975,Aloe guest house
178,178,ratnagiri,16.9833,73.3000,24.43,67,0,1.96,IN,1707073514,Vihar
192,192,nova sintra,14.8667,-24.7167,21.52,69,0,3.85,CV,1707072979,Residência Ka Dencho
224,224,salalah,17.0151,54.0924,23.72,43,0,2.13,OM,1707073521,Muscat International Hotel
244,244,cabo san lucas,22.8909,-109.9124,24.49,56,0,2.24,MX,1707073053,Comfort Rooms
259,259,antofagasta,-23.6500,-70.4000,27.60,66,7,2.93,CL,1707073021,Hotel Colón


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
%%capture --no-display

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = True,
    frame_width = 800,
    frame_height = 600,
    size= 80,
    cmap= 'rainbow',
    alpha= 0.6,
    hover_cols= ["Hotel Name", "Country", "City"],
    color= "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)